In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
!pip install keras_metrics
import keras_metrics
np.random.seed(1)
tf.random.set_seed(2)
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
%cd /content/
import tensorflow as tf
import numpy as np
from tensorflow import keras
!pip install keras_metrics
import keras_metrics
np.random.seed(1)
tf.random.set_seed(2)
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
%cd /content/
## split text file ##
with open('min-ind3-2.txt','r',encoding='utf-8') as f:
  lines = f.read().split('\n')
import sklearn
from sklearn.model_selection import train_test_split
train_val,test = train_test_split(lines,test_size=0.2) # here split size can be change with test_size parameter
with open('min-ind_test.txt','w',encoding='utf-8') as w:
  for line in test:
    w.write(line+'\n')

with open('min-ind_train.txt','w',encoding='utf-8') as w:
  for line in train_val:
    w.write(line+'\n')


kfold_acc = []
kf = KFold(n_splits=5,random_state=96, shuffle = True) # 5 folds for training and validation sets
save_dir = '/content/'
fold_var = 5
batch_size = 128
epochs = 10
latent_dim = 256 # edit LSTM units
num_samples = 14032
data_path = 'min-ind_train.txt'
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
  lines = f.read().split("\n")
  #print('lines',lines)
print('lines',lines[:len(lines)])
print(len(lines))
for line in lines[:num_samples]:
  if len(line.split(','))==2:
    input_text, target_text = line.split(",")
  # We use "tab" as the "start sequence" character
  # for the targets, and "\n" as "end sequence" character.
  target_text = "\t" + target_text + "\n"
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
      if char not in input_characters:
          input_characters.add(char)
  for char in target_text: # iterate over words in each line
      if char not in target_characters: 
          target_characters.add(char)
input_characters.add(':')
target_characters.add('-')
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) # whole vocabulary of characters
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts]) # maximum length of word 
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)]) # each char in input characters are encoded by index
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)]) # character to index dictionary

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


In [ ]:
## 5-fold BI-LSTM ##
latent_dim_1 = 128
latent_dim_2 = 256
latent_dim_3 = 512
import matplotlib.pyplot as plt
%matplotlib inline
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(input_texts)),decoder_target_data)):
  print('K={}'.format(i+1))
  encoder_inputs = keras.Input(shape=(None,num_encoder_tokens))
  encoder_outputs,forward_h,forward_c,backward_h,backward_c= keras.layers.Bidirectional(keras.layers.LSTM(latent_dim_2, return_state=True),merge_mode='concat',name='encoder_lstm1')(encoder_inputs)
  #encoder_outputs = encoder(encoder_inputs)
  #print(encoder)
  #encoder_outputs = keras.layers.Bidirectional(keras.layers.LSTM(latent_dim_2),merge_mode='concat',name='encoder_lstm2')(encoder)
  #print(encoder_outputs)
  #encoder_inputs = embedding_layer_input(encoder_inputs)
  # We discard `encoder_outputs` and only keep the states.
  state_h = tf.keras.layers.Concatenate()([forward_h,forward_c])
  state_c = tf.keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [state_h,state_c] # hidden and cell state
  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  #decoder_inputs = embedding_layer_output(decoder_inputs)
  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, return_sequences=True, return_state=True)(decoder_inputs, initial_state=encoder_states)
  #decoder_outputs, _, _ ,*_= decoder_lstm(decoder_inputs, initial_state=encoder_states)


  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_outputs) 

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  encoder_input_data_t = pd.Series(list(encoder_input_data)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data)).iloc[val_index].to_numpy()


  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])
  x_val = [encoder_input_data_val,decoder_input_data_val]
  y_val = decoder_target_data_val
  model.compile(
    optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  results = model.fit(
      [encoder_input_data_t, decoder_input_data_t],
      decoder_target_data_t,
      batch_size=batch_size,
      epochs=epochs,
      validation_data =(x_val,y_val)
  )
  plt.plot(results.history['accuracy'])
  plt.plot(results.history['val_accuracy'])
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend(['train','val'],loc='upper right')
  plt.title('Model Accuracy')
  plt.show()
  model.save('model_bilstm_256_{}-Fold{}'.format(fold_var,i+1))
  model = keras.models.load_model("model_bilstm_256_{}-Fold{}".format(fold_var,i+1)) # Using 5-Fold cross validated model
  encoder_inputs = model.input[0]  # input_1
  encoder_outputs,state_h_enc,state_c_enc,backward_h,backward_c  = model.layers[1].output  # lstm_1
  states_h = keras.layers.Concatenate()([state_h_enc,state_c_enc])
  states_c = keras.layers.Concatenate()([backward_h,backward_c])
  #print(states_h)
  encoder_states = [states_h,states_c]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim_3,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim_3,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
  decoder_lstm = model.layers[5]
  decoder_outputs, state_h_dec,state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs #, 
  )
  decoder_states = [state_h_dec,state_c_dec]
  decoder_dense = model.layers[6]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )
  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens,h,c = decoder_model.predict([target_seq] + states_value)
          # Sample a token
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char
          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True
          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0
          states_value = [h,c]
      return decoded_sentence
  save_dir = '/content/'
  #fold_var = 10
  #batch_size = 128
  #epochs = 25
  latent_dim = 256 # edit LSTM units
  num_samples = 14032
  data_path = 'min-ind_test.txt'
  input_texts = []
  target_texts = []
  #input_characters = set()
  #target_characters = set()
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  print('lines',lines[:len(lines)])
  print(len(lines))
  for line in lines[:num_samples]:
    if len(line.split(','))==2:
      input_text, target_text = line.split(",")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    
  '''
  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t-1, input_token_index[char]] = 1.0
  '''
  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t, input_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t, target_token_index[char]] = 1.0
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0

  
  df = pd.DataFrame()
  input_texts_ls = []
  decoded_sentences_ls = []
  target_texts_ls = []
  for seq_index in range(50):
      # Take one sequence (part of the training set)
      # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
      #print(input_seq)
    decoded_sentence= decode_sequence(input_seq)
    input_texts_ls.append(input_texts[seq_index].replace("\t", "").replace("\n", ""))
    target_texts_ls.append(target_texts[seq_index].replace("\t", "").replace("\n", ""))
    decoded_sentences_ls.append(decoded_sentence.replace("\t", "").replace("\n", ""))
    #print(type(input_texts_ls))
    #print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
    print("Target sentence:", target_texts[seq_index])
  temp_list = [input_texts_ls, target_texts_ls, decoded_sentences_ls]
  
  df = pd.DataFrame(temp_list).transpose()
  df = df.rename(columns={0: 'input_text', 1: 'target_text', 2:'decode_sentence'})
  print(df)
  df_csv = df.to_csv('Input_decoded_sentences_5-fold-{}.csv'.format(i))
  data_path = 'min-ind_train.txt'
  input_texts = []
  target_texts = []
  input_characters = set()
  target_characters = set()
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    #print('lines',lines)
  print('lines',lines[:len(lines)])
  print(len(lines))
  for line in lines[:num_samples]:
    if len(line.split(','))==2:
      input_text, target_text = line.split(",")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
  input_characters.add(':')
  target_characters.add('-')
  input_characters = sorted(list(input_characters))
  target_characters = sorted(list(target_characters))
  num_encoder_tokens = len(input_characters)
  num_decoder_tokens = len(target_characters)
  max_encoder_seq_length = max([len(txt) for txt in input_texts])
  max_decoder_seq_length = max([len(txt) for txt in target_texts])
  '''
  print("Number of samples:", len(input_texts))
  print("Number of unique input tokens:", num_encoder_tokens)
  print("Number of unique output tokens:", num_decoder_tokens)
  print("Max sequence length for inputs:", max_encoder_seq_length)
  print("Max sequence length for outputs:", max_decoder_seq_length)
  '''
  input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
  target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

  encoder_input_data = np.zeros(
      (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
  )
  decoder_input_data = np.zeros(
      (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
  )
  decoder_target_data = np.zeros(
      (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
  )

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t, input_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t, target_token_index[char]] = 1.0
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0

        
  #Save model
  #model.save("s2s_1024_f")